In [8]:
import pandas as pd
import sys

sys.path.append("../")

from lib.uid import gen_uid
from lib.clean import clean_names, clean_races, clean_sexes

In [9]:
def read_personnel_and_employment_hist():
    dfa = pd.read_csv("../../../data/FL/2023-4-13/employment-20231220144255.csv", encoding="latin1")

    dfb = pd.read_csv("../../../data/FL/2023-4-13/person_data-20231220144255.csv", encoding="latin1")

    df = pd.merge(dfa, dfb, on="person_nbr")

    dfc = pd.read_csv("../../../data/FL/2023-4-13/agency_map.csv", encoding="latin1")

    df = pd.merge(df, dfc, on="agcy_nbr", how="left")
    return df


In [10]:
df = read_personnel_and_employment_hist()

df = df.sort_values("person_nbr")
df = df.head(50)

df.to_csv("fl-post-review.csv", index=False)

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_2951/1656383686.py:2: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../../../data/FL/2023-4-13/employment-20231220144255.csv", encoding="latin1")


In [11]:
df = df.rename(columns={"sex_code": "sex", "race_code": "race", "employ_start_date": "hire_date", "separation_date": "left_date", "agcy_name": "agency"})

df = df.pipe(clean_names, ["first_name", "last_name", "middle_name"]).pipe(gen_uid, ["first_name", "last_name", "middle_name", "agency"])

def clean_dates(df):
    # Convert the 'ST_DATE' column to datetime
    df['hire_date'] = pd.to_datetime(df['hire_date'])
    
    # Format the dates as 'Month Day, Year'
    df['hire_date'] = df['hire_date'].dt.strftime('%m/%d/%Y')

    df['left_date'] = pd.to_datetime(df['left_date'])


    df['left_date'] = df['left_date'].dt.strftime('%m/%d/%Y')
    return df

df = df.pipe(clean_dates).pipe(clean_races, ["race"]).pipe(clean_sexes, ["sex"])

df["state"] = "FL"

standardize_from_lookup_table: unmatched sequences:
  {'oth'}
standardize_from_lookup_table: unmatched sequences:
  {'u'}


In [12]:
df

,employment_nbr,person_nbr,agcy_nbr,employ_type,employ_class,hire_date,left_date,separation_code,employ_tea,fingerprint_card_process_date,...,last_name,first_name,middle_name,sex,race,birth_year,Unnamed: 0,agency,uid,state
535437,416906,1,611,FT,Inst,12/04/2008,NaN,NaN,N,NaN,...,chance,lex,lane,male,white,1972.0,702,Florida Department Of Law Enforcement,c813b86baea6f22332c682f1902b68d0,FL
337201,252441,1,282,PT,LE,03/23/1999,01/01/2004,Vol,N,7/27/1999 0:00:00,...,chance,lex,lane,male,white,1972.0,481,Havana Police Department,dcb4eee28eec913f9e32e61d4d69ce69,FL
509656,400588,1,146,Aux,LE,12/17/2007,02/07/2008,Vol,N,11/19/2007 0:00:00,...,chance,lex,lane,male,white,1972.0,469,Jefferson County Sheriff's Office,3d1e29694a337fd1b1973a996bd96a0f,FL
513706,402530,1,11,FT,LE,02/07/2008,NaN,NaN,N,12/17/2007 0:00:00,...,chance,lex,lane,male,white,1972.0,312,Florida Department Of Law Enforcement,c813b86baea6f22332c682f1902b68d0,FL
273544,188790,2,455,PT,LE,02/11/1994,07/13/2001,TWA,N,2/11/1994 0:00:00,...,clendinen,stewart,,male,white,1961.0,689,Tallahassee Police Department,266103d305ce28adb8efb73f96525063,FL
379909,308104,2,611,FT,Inst,10/24/2000,11/06/2002,Vol,N,NaN,...,clendinen,stewart,,male,white,1961.0,702,Florida Department Of Law Enforcement,0094eb8d9918934038412663c3a40d27,FL
470423,474054,2,455,FT,Inst,02/15/2013,04/01/2021,MR,N,NaN,...,clendinen,stewart,,male,white,1961.0,689,Tallahassee Police Department,266103d305ce28adb8efb73f96525063,FL
188207,103442,2,488,FT,LE,04/22/1991,01/02/1994,Vol,N,4/22/1991 0:00:00,...,clendinen,stewart,,male,white,1961.0,374,Winter Park Police Department,94e09ae2b1ea3122ccdf578648838b81,FL
371539,286343,2,455,FT,LE,07/14/2001,NaN,NaN,N,3/5/2002 0:00:00,...,clendinen,stewart,,male,white,1961.0,689,Tallahassee Police Department,266103d305ce28adb8efb73f96525063,FL
392499,319014,2,668,FT,Inst,11/06/2002,02/14/2013,INST-REQ,N,NaN,...,clendinen,stewart,,male,white,1961.0,709,Fl Public Safety Institute Pat Thomas Law Enfo...,c29af4fbc346df95e64b05438e92df64,FL


In [13]:
# df_events = df[["uid", "agency", "state", "hire_date", "left_date"]]


# df_events = df_events[~((df_events.left_date.fillna("") == ""))]

# # Unpivot the DataFrame
# df_long = df_events.melt(id_vars=['uid', 'agency', 'state'], 
#                   value_vars=['hire_date', 'left_date'], 
#                   var_name='event_type', 
#                   value_name='event_date')

# # Generate event_uid using gen_uid function
# df_long = df_long.pipe(gen_uid, ['uid', 'event_type', 'event_date'], "event_uid")

# events_df= df_long[df_long.duplicated(subset=["uid"], keep=False)]

# # officers that we know have moved
# events_df.sort_values("uid")

# # events_df.to_csv("./../../../data/FL/match/events_florida_dropped_na_left_dates.csv", index=False)

In [14]:
# dup_uids_list = [x for x in events_df["uid"]]

# per_df = df[["first_name", "middle_name", "last_name", "sex", "race", "birth_year", "uid", "state"]]

# per_df = per_df[per_df.uid.isin(dup_uids_list)]

# # per_df.to_csv("./../../../data/FL/match/personnel_florida_dropped_na_left_dates.csv", index=False)